# Object detect

https://github.com/Megvii-BaseDetection/YOLOX


In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from PIL import Image
from tqdm import tqdm
from detector import *
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

***

### 1. detect target object and cut images

In [ ]:
data_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_tr = pd.DataFrame({'filepath':glob.glob(os.path.join(data_floder,'*.jpg'))})
df_tr['filename'] = df_tr['filepath'].map(lambda x: os.path.basename(x))

#TODO: cut test_public
#TODO: cut test_private


In [ ]:
model_path = '/mnt/hdd1/Model/Competition/orchid/yolox/last_epoch_ckpt.pth'
predictor = Predictor(conf=0.4, model_path=model_path)

aug_folder=data_floder+'_aug'
if not os.path.exists(aug_folder):
    os.makedirs(aug_folder)
    
for r in tqdm(df_tr.itertuples()):
    filepath= r.filepath
    img_raw = cv2.imread(filepath)
    img_h, img_w = img_raw.shape[:2]
    cls, scores, bboxes_tlbr, boxes_tlwh = predictor.detect(img_raw)
    if len(bboxes_tlbr)>0:
        bboxes_tlbr, valid_idx = predictor.parse_bbox(bboxes_tlbr)
        bboxes_tlbr = predictor.merge_bbox(bboxes_tlbr)
        x1,y1,x2,y2 = np.array(bboxes_tlbr).astype(int)
        x1 = np.clip(x1, 0, img_w)
        y1 = np.clip(y1, 0, img_h)
        x2 = np.clip(x2, 0, img_w)
        y2 = np.clip(y2, 0, img_h)        
        img_cut = img_raw[y1:y2,x1:x2]
    else:
        img_cut = img_raw
    img = Image.fromarray(img_cut)
    fname = 'aug_'+os.path.basename(filepath)
    save_path = os.path.join(aug_folder,fname)    
    img.save(save_path, quality=90)
    
ori_qty = len(glob.glob(os.path.join(data_floder,'*.jpg')))
aug_qty = len(glob.glob(os.path.join(aug_folder,'*.jpg')))
assert aug_qty == ori_qty, 'image qty not equal'    


***
### DEBUG

In [ ]:
        
def draw_boxes(image, box, adc, msg, color, thickness=8):
    xmin, ymin, xmax, ymax = list(map(int, box))
    label = f'{msg} score:{adc:.1f}'
    font_size=0.8
    ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_size, 2)
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, thickness)
    cv2.rectangle(image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
    cv2.putText(image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), 2)
    

In [ ]:
model_path = '/mnt/hdd1/Model/Competition/orchid/yolox/last_epoch_ckpt.pth'
predictor = Predictor(conf=0.4, model_path=model_path)


In [ ]:
data_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_tr = pd.DataFrame({'filepath':glob.glob(os.path.join(data_floder,'*.jpg'))})
df_tr['filename'] = df_tr['filepath'].map(lambda x: os.path.basename(x))

for i in range(1000, 1010):
    color = [211,199,178]
    filepath = df_tr.iloc[i]['filepath']
    img_raw = cv2.imread(filepath)
    cls, scores, bboxes_tlbr, boxes_tlwh = predictor.detect(img_raw)
    
    plt.figure(figsize=(12,6))
    plt.subplot(131)
    img_draw = img_raw.copy()
    for i in range(len(bboxes_tlbr)):
        draw_boxes(img_draw, bboxes_tlbr[i], scores[i], '', color)
    plt.imshow(img_draw[:,:,::-1])

    plt.subplot(132)
    if len(bboxes_tlbr)>0:
        bboxes_tlbr, valid_idx = predictor.parse_bbox(bboxes_tlbr)
        scores = scores[valid_idx]
    img_draw = img_raw.copy()
    for i in range(len(bboxes_tlbr)):
        draw_boxes(img_draw, bboxes_tlbr[i], scores[i], '', color)
    plt.imshow(img_draw[:,:,::-1])    
    
    plt.subplot(133)
    if len(bboxes_tlbr)>0:
        bboxes_tlbr = predictor.merge_bbox(bboxes_tlbr)

    img_draw = img_raw.copy()
    if len(bboxes_tlbr)>0:
        score = scores.mean()
        draw_boxes(img_draw, bboxes_tlbr, score, '', color)
    plt.imshow(img_draw[:,:,::-1])    
        
        
    plt.show()
    #print(cls, COCO_CLASSES[int(cls[0])], scores)

***